In [1]:
%display latex

eps = var('eps', latex_name = '\\varepsilon')
t = var('t')
r = var('r')
th = var('th', latex_name = '\\theta')
ph = var('ph', latex_name = '\\varphi')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne', latex_name='N_e')(r,th,eps)

In [2]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for kk in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for kk in range(0,4):
                chris[i][j][kk] = 0.5*sum(geuu[i,dd]*(diff(gedd[dd,j],coords[kk])
                                                   + diff(gedd[dd,kk],coords[j])
                                                   - diff(gedd[j,kk],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for kk in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for kk in range(0,4):
                for l in range(0,4):
                    riem[i][j][kk][l] = (diff(chris[i][j][l],coords[kk])
                                      - diff(chris[i][j][kk],coords[l])
                                      + sum(chris[i][kk][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][kk][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for kk in range(0,4):
            ricci[i,kk] = sum(riem[dd][i][dd][kk] for dd in range(0,4))

    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    Ge = matrix(SR, 4, 4)
    Ge = ricci - 0.5*gedd*s_curv
    
    dGe = matrix(SR, 4, 4)
    ddGe = matrix(SR, 4, 4)
    for i in range(0,4):
        for j in range(0,4):
            dGe[i,j] = diff(Ge[i,j],eps)
            ddGe[i,j] = diff(Ge[i,j],eps,eps)
    
    return {'inverse_metric': geuu,
            'einstein': Ge,
            'd_einstein': dGe,
            'dd_einstein': ddGe}

In [3]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(i,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]
        gedd[j,i] = gedd[i,j]

tensors = cosmotensors(gedd)

geuu = tensors['inverse_metric']
Ge = tensors['einstein']
dGe = tensors['d_einstein']
ddGe = tensors['dd_einstein']

In [4]:
show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

'ge0 = ' [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

'K1 = ' [                      0                       0                       0 -r^2*omega(r)*sin(th)^2]
[                      0                       0                       0                       0]
[                      0                       0                       0                       0]
[-r^2*omega(r)*sin(th)^2                       0                       0                       0]

'K2 = ' [2*r^2*omega(r)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                               0                                               0                                               0]
[                                              0                            4*e^lamb(r)*m(r, th)                                               0                                               0]
[                                              0                                               0                                  4*r^2*k(r, th)                                               0]
[                                              0                                               0                                               0                        4*r^2*k(r, th)*sin(th)^2]

'ge = ' [(r^2*omega(r)^2*sin(th)^2 - 2*e^nu(r)*h(r, th))*eps^2 - e^nu(r)                                                               0                                                               0                                     -eps*r^2*omega(r)*sin(th)^2]
[                                                              0                          2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                               0                                                               0]
[                                                              0                                                               0                                      2*eps^2*r^2*k(r, th) + r^2                                                               0]
[                                    -eps*r^2*omega(r)*sin(th)^2                                                               0                                                               0                  2*eps^2*r^2*k(r, th)*sin(th)^2 + r^2*sin(th)^2]

In [5]:
#4-velocities of the fluid for any epsilon

ue = [0,0,0,0]
u = [0,0,0,0]

ue[0] = Ne;         ue[1] = 0;   ue[2] = 0;   ue[3] = Ne*Omegae
u[0] = exp(-nu/2);  u[1] =  0;   u[2] = 0;    u[3] =  0

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                             #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                                     #Eq 4.20

In [6]:
show(LatexExpr("N_{\\varepsilon}\lvert_0 = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\lvert_0 = "), Omegaevalue)

N_{\varepsilon}\lvert_0 =  e^(-1/2*nu(r))

\Omega_{\varepsilon}\lvert_0 =  0

In [7]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10',latex_name="u^{(1)0}")(r,th)
u20 = function('u20',latex_name="u^{(2)0}")(r,th)

Omega1 = var('Omega1', latex_name="\\Omega^{(1)}")
Omega2 = var('Omega2', latex_name="\\Omega^{(2)}")

s0 = eps == 0

s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue

s3 = diff(Ne,eps,eps).subs(eps==0) == u20
s4 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

s5 = diff(Ne,eps).subs(eps==0) == u10
s6 = diff(Omegae,eps).subs(eps==0) == Omega1

sus_func = [s0,s1,s2,s3,s4,s5,s6]

for i in range(0,4):
    u1[i] = diff(ue[i],eps)
    u2[i] = diff(ue[i],eps,eps)

Ue = [0,0,0,0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[j]*gedd[j,i]

geUeue = 0
for i in range(0,4):
    geUeue += Ue[i]*ue[i]

ddgeUeue = diff(geUeue, eps, eps).subs(sus_func) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s7 = u20 == u20value

dgeUeue = diff(geUeue, eps).subs(sus_func) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s8 = u10 == u10value

sus_func.extend([s7,s8])
for i in range(0,4):
    for kk in range(0,len(sus_func)):
        u1[i] = u1[i].subs(sus_func[kk])
        u2[i] = u2[i].subs(sus_func[kk])

In [8]:
show(LatexExpr("u^{(1)} = "), u1)                                                                         #Eq 4.21
show(LatexExpr("u^{(2)} = "), u2)                                                                         #Eq 4.22

u^{(1)} =  [0, 0, 0, Omega1*e^(-1/2*nu(r))]

u^{(2)} =  [((Omega1^2*r^2 - 2*Omega1*r^2*omega(r) + r^2*omega(r)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)),
 0,
 0,
 Omega2*e^(-1/2*nu(r))]

In [9]:
# CHECK
u1_paper = [0,0,0,0]
u1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
u1_paper[3] = e^(-nu/2)*Omega1

u2_paper = [0,0,0,0]
u2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omega1*K1[0,3]+Omega1^2*r^2*sin(th)^2)
u2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omega1+Omega2)  

u1_sum = 0
u2_sum = 0
for i in range(0,4):
    u1_sum += u1[0] - u1_paper[0]
    u2_sum += u2[0] - u2_paper[0]

u1_sum = u1_sum.simplify_full()
u2_sum = u2_sum.simplify_full()

show(u1_sum)
show(u2_sum)

0

0

In [10]:
G = copy(Ge)
for kk in range(0,len(sus_func)):
    G = G.subs(sus_func[kk])

In [11]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,th,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,th,eps)

Te = matrix(SR, 4, 4)
dTe = matrix(SR, 4, 4)
ddTe = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        Te[i,j] = (Ee+Pe)*Ue[i]*Ue[j]+Pe*gedd[i,j]
        dTe[i,j] = diff(Te[i,j],eps)
        ddTe[i,j] = diff(Te[i,j],eps,eps)

T = copy(Te)
for kk in range(0,len(sus_func)):
    T = T.subs(sus_func[kk])

In [12]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), T)

T_{\varepsilon}\lvert_0 = [(Ee(r, th, 0) + Pe(r, th, 0))*e^nu(r) - Pe(r, th, 0)*e^nu(r)                                                            0                                                            0                                                            0]
[                                                           0                                       Pe(r, th, 0)*e^lamb(r)                                                            0                                                            0]
[                                                           0                                                            0                                             r^2*Pe(r, th, 0)                                                            0]
[                                                           0                                                            0                                                            0                                   r^2*Pe(r, th, 0)*sin(th)^2]

In [13]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sus_func.extend([s9,s10])

T = T.subs(s9).subs(s10)
        
Evalue = solve(G[0,0]==kappa*T[0,0],E)[0].rhs()                                                          #Eq 3.7
Pvalue = solve(G[1,1]==kappa*T[1,1],P)[0].rhs()                                                          #Eq 3.8
Pvalue2 = solve(G[2,2]==kappa*T[2,2],P)[0].rhs()
Pvalue3 = solve(G[3,3]==kappa*T[3,3],P)[0].rhs()
ddnu = solve(Pvalue == Pvalue2, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [14]:
show(LatexExpr("E = "), Evalue)
show(LatexExpr("P_{rr} = "), Pvalue)
show(LatexExpr("\\nu'' ="), ddnu)

s11 = E == Evalue
s12 = P == Pvalue
s13 = diff(nu,r,r) == ddnu
sus_func.extend([s11,s12,s13])
sus_func.extend([diff(E,r)==diff(Evalue,r),diff(P,r)==diff(Pvalue,r)])

E =  1/8*(r*e^(-lamb(r))*diff(lamb(r), r) - e^(-lamb(r)) + 1)/(pi*r^2)

P_{rr} =  1/8*(r*e^(-lamb(r))*diff(nu(r), r) + e^(-lamb(r)) - 1)/(pi*r^2)

\nu'' = -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2

In [15]:
Evalue_paper = diff(lamb,r) - 1/r*(1-e^(lamb)) - r*e^(lamb)*8*pi*E == 0
Evalue_paper = solve(Evalue_paper, E)[0].rhs()

Evalue_check = Evalue - Evalue_paper
show(Evalue_check.simplify_full())

Pvalue_paper = diff(nu,r) - 1/r*(-1+e^(lamb)) - r*e^(lamb)*8*pi*P == 0
Pvalue_paper = solve(Pvalue_paper, P)[0].rhs()

Pvalue_check = Pvalue - Pvalue_paper
show(Pvalue_check.simplify_full())

ddnu_paper = 2*r*diff(nu,r,r) + diff(nu,r)*(r*diff(nu,r)-2)-diff(lamb,r)*(2+r*diff(nu,r))+4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, diff(nu,r,r))[0].rhs()

ddnu_check = ddnu - ddnu_paper
show(ddnu_check.simplify_full())

0

0

0

## Primera derivada

In [16]:
E1 = function('E1', latex_name='E^{(1)}')(r,th)
P1 = function('P1', latex_name='P^{(1)}')(r,th)

In [17]:
s14 = diff(Ee,eps).subs(eps==0) == E1
s15 = diff(Pe,eps).subs(eps==0) == P1

sus_func.extend([s14,s15])

In [18]:
dG = copy(dGe)
dT = copy(dTe)
for kk in range(0,len(sus_func)):
    dG = dG.subs(sus_func[kk])
    dT = dT.subs(sus_func[kk])

In [19]:
dEq11 = dG[1,1] - kappa*dT[1,1] == 0
dEq11 = dEq11.simplify_full()

dEq22 = dG[2,2] - kappa*dT[2,2] == 0
dEq22 = dEq22.simplify_full()

dEq33 = dG[3,3] - kappa*dT[3,3] == 0
dEq33 = dEq33.simplify_full()

show(dEq11)
show(dEq22)
show(dEq33)

P1value = solve(dEq11,P1)[0].rhs()
sus_func.extend([P1==P1value])

show(LatexExpr("\\text{De aquí sacamos }P_1(r) \\rightarrow P_1(r) = "), P1value)

-8*pi*P1(r, th)*e^lamb(r) == 0

-8*pi*r^2*P1(r, th) == 0

-8*pi*r^2*P1(r, th)*sin(th)^2 == 0

\text{De aquí sacamos }P_1(r) \rightarrow P_1(r) =  0

In [20]:
dEq00 = dG[0,0] - kappa*dT[0,0] == 0
dEq00 = dEq00.simplify_full()
show(dEq00)

for kk in range(0,len(sus_func)):
    dEq00 = dEq00.subs(sus_func[kk])

E1value = solve(dEq00,E1)[0].rhs()
sus_func.extend([E1==E1value])

show(LatexExpr("\\text{De aquí sacamos }E_1(r,\\theta) \\rightarrow E_1(r,\\theta) = "), E1value)

-8*pi*E1(r, th)*e^nu(r) == 0

\text{De aquí sacamos }E_1(r,\theta) \rightarrow E_1(r,\theta) =  0

In [21]:
dEq03 = dG[0,3] - kappa*dT[0,3]

for kk in range(0,len(sus_func)):
    dEq03 = dEq03.subs(sus_func[kk])
    
dEq03 = dEq03.simplify_full()

ddomega = solve(dEq03, diff(omega,r,r))[0].rhs()
ddomega = ddomega.simplify_full()
sus_func.extend([diff(omega,r,r)==ddomega])

In [22]:
# CHECK EQUATION 48
jj = e^(-(lamb+nu)/2)
dEq03_paper = diff(r^4*jj*diff(omega,r),r)+\
              (r^2*jj*e^(lamb)/sin(th)^3)*diff(sin(th)^3*diff(omega,th),th)+\
              4*r^3*diff(jj,r)*(omega-Omega1)

dEq03_paper = dEq03_paper*sin(th)^2/(2*r^2*e^(lamb/2-nu/2))

dEq03_check = dEq03_paper - dEq03

dEq03_check.simplify_full()

0

## Segunda derivada

In [23]:
E2 = function('E2', latex_name='E^{(2)}')(r,th)
P2 = function('P2', latex_name='P^{(2)}')(r,th)

In [24]:
s16 = diff(Ee,eps,eps).subs(eps==0) == E2
s17 = diff(Pe,eps,eps).subs(eps==0) == P2

sus_func.extend([s16,s17])

In [25]:
ddG = copy(ddGe)
ddT = copy(ddTe)
for kk in range(0,len(sus_func)):
    ddG = ddG.subs(sus_func[kk])
    ddT = ddT.subs(sus_func[kk])

### Descomposición en polinomios de Legendre

In [26]:
h0 = function('h0', latex_name='h_0')(r)
k0 = function('k0', latex_name='k_0')(r)
m0 = function('m0', latex_name='m_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
m2 = function('m2', latex_name='m_2')(r)

E20 = function('E20', latex_name='E_{0}^{(2)}')(r)
P20 = function('P20', latex_name='P_{0}^{(2)}')(r)

E22 = function('E22', latex_name='E_{2}^{(2)}')(r)
P22 = function('P22', latex_name='P_{2}^{(2)}')(r)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [27]:
sus_descomp = [m==m0+m2*lp2,
               diff(m,r)==diff(m0+m2*lp2,r),
               diff(m,r,r)==diff(m0+m2*lp2,r,r),
               diff(m,th)==diff(m0+m2*lp2,th),
               diff(m,th,th)==diff(m0+m2*lp2,th,th),
               diff(m,th,r)==diff(m0+m2*lp2,th,r),
               diff(m,r,th)==diff(m0+m2*lp2,r,th),
               k==k2*lp2,
               diff(k,r)==diff(k2*lp2,r),
               diff(k,r,r)==diff(k2*lp2,r,r),
               diff(k,th)==diff(k2*lp2,th),
               diff(k,th,th)==diff(k2*lp2,th,th),
               diff(k,th,r)==diff(k2*lp2,th,r),
               diff(k,r,th)==diff(k2*lp2,r,th),
               h==h0+h2*lp2,
               diff(h,r)==diff(h0+h2*lp2,r),
               diff(h,r,r)==diff(h0+h2*lp2,r,r),
               diff(h,th)==diff(h0+h2*lp2,th),
               diff(h,th,th)==diff(h0+h2*lp2,th,th),
               diff(h,th,r)==diff(h0+h2*lp2,th,r),
               diff(h,r,th)==diff(h0+h2*lp2,r,th),
               E2==E20+E22*lp2,
               diff(E2,r)==diff(E20+E22*lp2,r),
               diff(E2,r,r)==diff(E20+E22*lp2,r,r),
               diff(E2,th)==diff(E20+E22*lp2,th),
               diff(E2,th,th)==diff(E20+E22*lp2,th,th),
               diff(E2,th,r)==diff(E20+E22*lp2,th,r),
               diff(E2,r,th)==diff(E20+E22*lp2,r,th),
               P2==P20+P22*lp2,
               diff(P2,r)==diff(P20+P22*lp2,r),
               diff(P2,r,r)==diff(P20+P22*lp2,r,r),
               diff(P2,th)==diff(P20+P22*lp2,th),
               diff(P2,th,th)==diff(P20+P22*lp2,th,th),
               diff(P2,th,r)==diff(P20+P22*lp2,th,r),
               diff(P2,r,th)==diff(P20+P22*lp2,r,th)]

for kk in range(0,len(sus_descomp)):
    ddG = ddG.subs(sus_descomp[kk])
    ddT = ddT.subs(sus_descomp[kk])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()

# l=0

In [28]:
s1 = sin(th)^2 == (2/3)*(1-LP2)
s2 = cos(th)^2 == (2*LP2+1)/3
s3 = cos(th) == LP1

sus_legendre = [s1,s2,s3]

In [29]:
ddEq03 = ddG[0,3] - kappa*ddT[0,3] == 0
ddEq03 = ddEq03.simplify_full()

for kk in range(0,len(sus_legendre)):
    ddEq03 = ddEq03.subs(sus_legendre[kk])
ddEq03 = ddEq03.subs(LP1==0).subs(LP2==0)

Omega2value = solve(ddEq03,Omega2)[0].rhs()
show(LatexExpr("\\Omega^{(2)} = "), Omega2value)

ddG = ddG.subs(Omega2==Omega2value)
ddT = ddT.subs(Omega2==Omega2value)
sus_func.extend([Omega2==Omega2value])

\Omega^{(2)} =  0

In [31]:
ddEq00 = ddG[0,0] - kappa*ddT[0,0] == 0
ddEq00 = ddEq00.simplify_full()
E20value = solve(ddEq00,E20)[0].rhs()

for kk in range(0,len(sus_legendre)):
    E20value = E20value.subs(sus_legendre[kk])

E20value = E20value.subs(LP1==0).subs(LP2==0)
E20value = E20value.simplify_full()

ddG = ddG.subs(E20==E20value)
ddT = ddT.subs(E20==E20value)
sus_func.extend([E20==E20value])
    
E20value_paper = (4/r^2*diff(r*e^(-lamb)*m0,r)\
                  +8/3*r*jj*diff(jj,r)*(omega-Omega1)^2\
                  -1/3*jj^2*r^2*diff(omega,r)^2)/(8*pi)

E20value_paper = E20value_paper.subs(Omega2==Omega2value)

E20value_check = E20value - E20value_paper
E20value_check = E20value_check.simplify_full()

show(E20value_check)

0

In [32]:
ddEq11 = ddG[1,1] - kappa*ddT[1,1] == 0
ddEq11 = ddEq11.simplify_full()
P20value = solve(ddEq11,P20)[0].rhs()
P20value = P20value.simplify_full()

for kk in range(0,len(sus_legendre)):
    P20value = P20value.subs(sus_legendre[kk])
    
P20value = P20value.subs(LP1==0).subs(LP2==0)
P20value = P20value.simplify_full()

ddG = ddG.subs(P20==P20value)
ddT = ddT.subs(P20==P20value)
sus_func.extend([P20==P20value])

P20value_paper = (4/r^2*(r*e^(-lamb)*diff(h0,r)-m0*(8*pi*r^2*P+1))\
                  +1/3*r^2*jj^2*diff(omega,r)^2)/(8*pi)

for kk in range(0,len(sus_func)):
    P20value_paper = P20value_paper.subs(sus_func[kk])
P20value_paper = P20value_paper.simplify_full()

P20value_check = P20value - P20value_paper
P20value_check = P20value_check.simplify_full()

show(P20value_check)

0

In [33]:
ddEq22 = ddG[2,2] - kappa*ddT[2,2] == 0
ddEq22 = ddEq22.simplify_full()
ddh0 = solve(ddEq22,diff(h0,r,r))[0].rhs()
ddh0 = ddh0.simplify_full()

for kk in range(0,len(sus_legendre)):
    ddh0 = ddh0.subs(sus_legendre[kk])
    
ddh0 = ddh0.subs(LP1==0).subs(LP2==0)
ddh0 = ddh0.simplify_full()

ddEq33 = ddG[3,3] - kappa*ddT[3,3] == 0
ddEq33 = ddEq33.simplify_full()
ddh0_33 = solve(ddEq33,diff(h0,r,r))[0].rhs()
ddh0_33 = ddh0_33.simplify_full()

for kk in range(0,len(sus_legendre)):
    ddh0_33 = ddh0_33.subs(sus_legendre[kk])
    
ddh0_33 = ddh0_33.subs(LP1==0).subs(LP2==0)
ddh0_33 = ddh0_33.simplify_full()

ddG = ddG.subs(diff(h0,r,r)==ddh0)
ddT = ddT.subs(diff(h0,r,r)==ddh0)
sus_func.extend([diff(h0,r,r)==ddh0])

m2h2 = ddh0 - ddh0_33 == 0
m2value = solve(m2h2,m2)[0].rhs()
m2value = m2value.simplify_full()

m2value_paper = 1/6*r^4*jj^2*diff(omega,r)^2\
                -1/3*r^3*diff(jj^2,r)*(omega-Omega1)^2\
                -h2

for kk in range(0,len(sus_func)):
    m2value_paper = m2value_paper.subs(sus_func[kk])

ddG = ddG.subs(m2==m2value)
ddT = ddT.subs(m2==m2value)
sus_func.extend([m2==m2value])

m2value_check = m2value - m2value_paper
m2value_check = m2value_check.simplify_full()

m2value_check

0

In [34]:
gamma = var('gamma')
Pt0 = P20/(2*(E+P))

eq_64 = Pt0 + h0 - 1/3*r^2*e^(-nu)*(omega-Omega1)^2 - gamma == 0
for kk in range(0,len(sus_func)):
    eq_64 = eq_64.subs(sus_func[kk])

deq_64 = diff(eq_64,r).simplify_full()
ddh0_64 = solve(deq_64, diff(h0,r,r))[0].rhs()
ddh0_64 = ddh0_64.simplify_full()

for kk in range(0,len(sus_func)):
    ddh0_64 = ddh0_64.subs(sus_func[kk])
    ddh0 = ddh0.subs(sus_func[kk])
    
ddh0_64 = ddh0_64.simplify_full()
ddh0 = ddh0.simplify_full()

ddh0_check = ddh0 - ddh0_64
ddh0_check = ddh0_check.simplify_full()

show(ddh0)
show(ddh0_64)

1/6*(3*r^4*diff(omega(r), r)^2 + 6*r*e^nu(r)*diff(h0(r), r) + 6*r*e^nu(r)*diff(m0(r), r) - 12*e^(lamb(r) + nu(r))*m0(r) + (2*Omega1^2*r^3 - 4*Omega1*r^3*omega(r) + 2*r^3*omega(r)^2 + 3*r^2*e^nu(r)*diff(h0(r), r))*diff(lamb(r), r) + (2*Omega1^2*r^3 - 4*Omega1*r^3*omega(r) + 2*r^3*omega(r)^2 - 6*r^2*e^nu(r)*diff(h0(r), r) + 3*r^2*e^nu(r)*diff(m0(r), r))*diff(nu(r), r))*e^(-nu(r))/r^2

1/24*(24*r^2*e^nu(r)*diff(lamb(r), r)*diff(m0(r), r) - 4*(Omega1^2*r^5 - 2*Omega1*r^5*omega(r) + r^5*omega(r)^2)*diff(nu(r), r)^3 + 48*(e^lamb(r) - 1)*e^nu(r)*m0(r) - 48*(r*e^lamb(r) - r)*e^nu(r)*diff(h0(r), r) + 4*(2*Omega1^2*r^4 - 4*Omega1*r^4*omega(r) + 2*r^4*omega(r)^2 - 3*r^3*e^nu(r)*diff(h0(r), r) + 6*r^2*e^nu(r)*m0(r))*diff(lamb(r), r)^2 + 4*(2*Omega1^2*r^4 - 4*Omega1*r^4*omega(r) + 2*r^4*omega(r)^2 - 6*r^3*e^nu(r)*diff(h0(r), r) + 6*r^3*e^nu(r)*diff(m0(r), r) + 3*r^2*e^nu(r)*m0(r) - (2*Omega1^2*r^5 - 4*Omega1*r^5*omega(r) + 2*r^5*omega(r)^2 + 3*r^3*e^nu(r)*m0(r))*diff(lamb(r), r))*diff(nu(r), r)^2 - (2*r^6*diff(lamb(r), r)^2 + r^6*diff(nu(r), r)^2 - 2*r^6*diff(lamb(r), r, r) - 12*r^5*diff(lamb(r), r) + 4*r^4*e^lamb(r) - 4*r^4 + (r^6*diff(lamb(r), r) - 12*r^5)*diff(nu(r), r))*diff(omega(r), r)^2 + 24*(r^2*e^nu(r)*diff(h0(r), r) - 2*r*e^(lamb(r) + nu(r))*m0(r))*diff(lamb(r), r) + 24*(r^3*e^nu(r)*diff(h0(r), r) - r^2*e^nu(r)*m0(r))*diff(lamb(r), r, r) - 4*(6*r^3*e^nu(r)*m0(r)*diff(lamb(r), r, r) - 6*r^2*e^nu(r)*diff(h0(r), r) + 12*r*e^nu(r)*m0(r) + (Omega1^2*r^5 - 2*Omega1*r^5*omega(r) + r^5*omega(r)^2 - 3*r^3*e^nu(r)*m0(r))*diff(lamb(r), r)^2 - (4*Omega1^2*r^4 - 8*Omega1*r^4*omega(r) + 4*r^4*omega(r)^2 - 3*r^3*e^nu(r)*diff(h0(r), r) + 3*r^2*e^nu(r)*m0(r))*diff(lamb(r), r) - 6*(r^3*e^nu(r)*diff(lamb(r), r) + r^2*e^nu(r))*diff(m0(r), r))*diff(nu(r), r))/(r^3*e^nu(r)*diff(lamb(r), r) + r^3*e^nu(r)*diff(nu(r), r))

In [35]:
#Baro = E2*diff(P,r) - P2*diff(E,r) == 0
#for kk in range(0,len(sus_func)):
#    Baro = Baro.subs(sus_func[kk])
#    
#Baro

# l=2

In [36]:
# SACAR DIRECTAMENTE E22

ddEq00 = ddG[0,0] - kappa*ddT[0,0] == 0
ddEq00 = ddEq00.simplify_full()

for kk in range(0,len(sus_legendre)):
    ddEq00 = ddEq00.subs(sus_legendre[kk])

ddEq00 = ddEq00.subs(LP1==0) - ddEq00.subs(LP1==0).subs(LP2==0)
ddEq00 = ddEq00/LP2
ddEq00 = ddEq00.simplify_full()

E22value = solve(ddEq00,E22)[0].rhs()
for kk in range(0,len(sus_func)):
    E22value = E22value.subs(sus_func[kk])

E22value = E22value.simplify_full()
show(E22value)
# METERLO EN DDG, DDT Y EN SUS_FUNC

#E22value_paper = 4*diff(E,r)/3/diff(nu,r)*(3*h2+e^(-nu)*r^2*(omega-Omega1)^2)
#for kk in range(0,len(sus_func)):
#    E22value_paper = E22value_paper.subs(sus_func[kk])
#
#E22value_check = E22value - E22value_paper
#E22value_check = E22value_check.simplify_full()
#
#show(E22value_paper)

-1/24*(12*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) + 12*(3*e^(2*lamb(r)) + e^lamb(r))*e^nu(r)*h2(r) + 36*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r)^2 - 12*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + (2*r^5*diff(lamb(r), r) - 7*r^4*e^lamb(r) - 2*r^4)*diff(omega(r), r)^2 - 24*e^(2*lamb(r) + nu(r))*k2(r) - 2*(8*Omega1^2*r^3*e^lamb(r) + 2*Omega1^2*r^3 + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 6*r*e^(lamb(r) + nu(r))*h2(r) + 2*(4*r^3*e^lamb(r) + r^3)*omega(r)^2 - 4*(4*Omega1*r^3*e^lamb(r) + Omega1*r^3)*omega(r))*diff(lamb(r), r) - 4*(4*Omega1^2*r^3*e^lamb(r) + Omega1^2*r^3 + (4*r^3*e^lamb(r) + r^3)*omega(r)^2 - 2*(4*Omega1*r^3*e^lamb(r) + Omega1*r^3)*omega(r) - (Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*e^(-2*lamb(r) - nu(r))/(pi*r^2)

In [37]:
ddEq11 = ddG[1,1] - kappa*ddT[1,1] == 0
ddEq11 = ddEq11.simplify_full()

for kk in range(0,len(sus_legendre)):
    ddEq11 = ddEq11.subs(sus_legendre[kk])

ddEq11 = ddEq11.subs(LP1==0) - ddEq11.subs(LP1==0).subs(LP2==0)
ddEq11 = ddEq11/LP2
ddEq11 = ddEq11.simplify_full()

P22value = solve(ddEq11,P22)[0].rhs()
for kk in range(0,len(sus_func)):
    P22value = P22value.subs(sus_func[kk])
    
P22value = P22value.simplify_full()
show(P22value)
# METERLO EN DDG, DDT Y EN SUS_FUNC


#M = r/2*(1-e^(-lamb))
#
#dh2_paper = (-diff(nu,r)+r/((r-2*M)*diff(nu,r))*(8*pi*(E+P)-4*M/r^3))*h2\
#            -4*(h2+k2)/(r*diff(nu,r)*(r-2*M))\
#            +1/6*(1/2*r*diff(nu,r)-1/((r-2*M)*diff(nu,r)))*r^3*jj^2*diff(omega,r)^2\
#            -1/3*(1/2*r*diff(nu,r)+1/((r-2*M)*diff(nu,r)))*r^2*diff(jj^2,r)*(omega-Omega1)^2
#
#for kk in range(0,len(sus_func)):
#    dh2 = dh2.subs(sus_func[kk])
#    dh2_paper = dh2_paper.subs(sus_func[kk])
#
#dh2_check = dh2 - dh2_paper
#dh2_check = dh2_check.simplify_full()
#
#show(dh2_check)

-1/24*(12*(3*e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 12*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 12*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + (2*r^5*diff(nu(r), r) + r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + 24*e^(2*lamb(r) + nu(r))*k2(r) + 4*(Omega1^2*r^3 - 2*Omega1*r^3*omega(r) + r^3*omega(r)^2)*diff(lamb(r), r) + 2*(2*Omega1^2*r^3 - 4*Omega1*r^3*omega(r) + 2*r^3*omega(r)^2 - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) - 6*r*e^(lamb(r) + nu(r))*h2(r) + 2*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*e^(-2*lamb(r) - nu(r))/(pi*r^2)

In [38]:
ddEq12 = ddG[1,2]-kappa*ddT[1,2] == 0
ddEq12 = ddEq12.simplify_full()
h2value = solve(ddEq12, h2)[0].rhs()
h2value = h2value.simplify_full()

h2value_paper = (-diff(h2,r)-diff(k2,r)+(1/r+diff(nu,r)/2)*(-2/3*r^3*jj*diff(jj,r)*(omega-Omega1)^2+1/6*jj^2*r^4*diff(omega,r)^2))/diff(nu,r)

for kk in range(0,len(sus_func)):
    h2value = h2value.subs(sus_func[kk])
    h2value_paper = h2value_paper.subs(sus_func[kk])
    
h2value = h2value.simplify_full()
h2value_paper = h2value_paper.simplify_full()
h2value_check = h2value - h2value_paper
h2value_check = h2value_check.simplify_full()
show(h2value_check)

0

In [39]:
ddEq00 = ddG[2,2] - kappa*ddT[2,2] == 0
ddEq00 = ddEq00.simplify_full(); ddEq00

-1/6*(48*pi*r^2*P22(r)*e^(2*lamb(r) + nu(r)) - 12*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r, r) - 12*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) + 24*(e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 3*(24*pi*r^2*P22(r)*e^(2*lamb(r) + nu(r)) - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r, r) - 6*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) + 12*(e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 6*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 12*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 6*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + (2*r^5*diff(nu(r), r) - r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 + 3*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r))*diff(lamb(r), r) + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(m2(r), r) - 12*r*e^(lamb(r) + nu(r))*h2(r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*sin(th)^2 - 12*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 24*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 12*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + 8*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + 2*(2*r^5*diff(nu(r), r) - r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + 2*(4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 + 3*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r))*diff(lamb(r), r) + 2*(4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(m2(r), r) - 12*r*e^(lamb(r) + nu(r))*h2(r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*e^(-2*lamb(r) - nu(r)) == 0

In [40]:
ddEq00 = ddG[3,3] - kappa*ddT[3,3] == 0
ddEq00 = ddEq00.simplify_full(); ddEq00

1/6*(3*(24*pi*r^2*P22(r)*e^(2*lamb(r) + nu(r)) - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r, r) - 6*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) + 12*(e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 6*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 12*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 6*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + (2*r^5*diff(nu(r), r) - r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 + 3*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r))*diff(lamb(r), r) + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(m2(r), r) - 12*r*e^(lamb(r) + nu(r))*h2(r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*cos(th)^4 + 24*pi*r^2*P22(r)*e^(2*lamb(r) + nu(r)) - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r, r) - 6*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) - 4*(24*pi*r^2*P22(r)*e^(2*lamb(r) + nu(r)) - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r, r) - 6*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r, r) + 12*(e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 6*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 12*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 6*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + (2*r^5*diff(nu(r), r) - r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 + 3*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r))*diff(lamb(r), r) + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(m2(r), r) - 12*r*e^(lamb(r) + nu(r))*h2(r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*cos(th)^2 + 12*(e^(2*lamb(r)) - e^lamb(r))*e^nu(r)*h2(r) - 6*r*e^(lamb(r) + nu(r))*diff(h2(r), r) - 12*r*e^(lamb(r) + nu(r))*diff(k2(r), r) + 6*r*e^(lamb(r) + nu(r))*diff(m2(r), r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(nu(r), r)^2 + (2*r^5*diff(nu(r), r) - r^4*e^lamb(r) + 2*r^4)*diff(omega(r), r)^2 + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 + 3*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r))*diff(lamb(r), r) + (4*Omega1^2*r^3 - 8*Omega1*r^3*omega(r) + 4*r^3*omega(r)^2 - 6*r^2*e^(lamb(r) + nu(r))*diff(h2(r), r) - 3*r^2*e^(lamb(r) + nu(r))*diff(k2(r), r) + 3*r^2*e^(lamb(r) + nu(r))*diff(m2(r), r) - 12*r*e^(lamb(r) + nu(r))*h2(r) + 4*(Omega1^2*r^4 - 2*Omega1*r^4*omega(r) + r^4*omega(r)^2)*diff(lamb(r), r))*diff(nu(r), r))*e^(-2*lamb(r) - nu(r)) == 0

In [41]:
# ECUACION 67
## dh2k2value = -diff(nu,r)*h2+(1/r+diff(nu,r)/2)*(-2/3*r^3*jj*diff(jj,r)*(omega-Omega1)^2+1/6*jj^2*r^4*diff(omega,r)^2)
